In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, root_scalar
from sklearn.metrics import r2_score

In [34]:
def complexityN0(n_data, t_dijkstra, t_bmssp):
    def model_dijkstra(n, c):
        return c * (n * np.log10(n))

    def model_bmspp(n, c):
        return c * (n * np.power(np.log10(n), 2/3))
    
    params_d, _ = curve_fit(model_dijkstra, n_data, t_dijkstra, bounds=(0, np.inf))
    params_b, _ = curve_fit(model_bmspp, n_data, t_bmssp, bounds=(0, np.inf))

    c_d = params_d[0]
    c_b = params_b[0]

    if c_d == 0 or c_b == 0:
        return -1
    
    log_n_intersect = np.power(c_b / c_d, 3)
    
    return log_n_intersect

In [35]:
def powerLawN0(n_data, t_dijkstra, t_bmssp):
    # 1. Define model
    def model_power_law(n, c, a):
        return c * np.power(n, a)

    p0_power = [1e-9, 1.0]
    try:
        params_d_pow, _ = curve_fit(model_power_law, n_data, t_dijkstra, p0=p0_power, bounds=(0, np.inf), maxfev=10000)
        params_b_pow, _ = curve_fit(model_power_law, n_data, t_bmssp, p0=p0_power, bounds=(0, np.inf), maxfev=10000)
    except RuntimeError:
        return -1 # Fit failed

    cd, ad = params_d_pow
    cb, ab = params_b_pow

    # 3. Analytic Solution
    # Equation: log(n) = (log(cb) - log(cd)) / (ad - ab)
    
    # Avoid division by zero if exponents are identical
    if np.isclose(ad, ab):
        return -1 
        
    numerator = np.log10(cb) - np.log10(cd)
    denominator = ad - ab
    
    log_n_intersect = numerator / denominator
    
    return log_n_intersect

In [36]:
graph_types = ['GridR', 'GridED', 'H3', 'D3', 'USA']

print("Graph Type ComplexityN0 PowerLawN0")
for graph_type in graph_types:
    df = pd.read_csv(f'outputs/results_{graph_type}/performance_summary_table.csv')
    df = df[df['Número de Vértices'] >= 100].copy()
    n_data = df['Número de Vértices'].values
    t_dijkstra = df['Tempo Dijkstra (ms)'].values / 1000
    t_bmssp = df['Tempo BMSPP (ms)'].values / 1000
    print(graph_type, np.round(complexityN0(n_data, t_dijkstra, t_bmssp)), np.round(powerLawN0(n_data, t_dijkstra, t_bmssp)))


Graph Type ComplexityN0 PowerLawN0
GridR 290.0 15.0
GridED 1482.0 16.0
H3 383.0 4.0
D3 370.0 3.0
USA 514.0 -4.0
